In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
import os
import tensorflow as tf
import numpy as np
import pathlib
from matplotlib import pyplot as plt
import pandas as pd
import glob
from scipy.fft import fft, ifft
import math

In [9]:
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.eperimental.set_memory_growth(gpu, True)

In [10]:
data = '../input/first-arrivals-ml-in-physics/data/train'
data_dir = os.listdir(data)

In [11]:
import re
def numericalSort(value):
    numbers = re.compile(r'(\d+)')
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

In [12]:
df =sorted(glob.glob( os.path.join(data, "simulation_*/receiver_0.csv")), key=numericalSort)
df = pd.concat(map(pd.read_csv, df), ignore_index=True)
df

In [13]:
df_output = sorted(glob.glob( os.path.join(data, "simulation_*_output.csv")), key=numericalSort)
df_output = pd.concat(map(pd.read_csv, df_output), ignore_index=True)
# df_output = df_output.replace(-1, np.nan)
df_output

In [14]:
def to_fft(waveform):
    convert = np.fft.fft(waveform)
    convert = tf.abs(convert)
    return convert

In [15]:
# X_fft = to_fft(df.X)
# Y_fft = to_fft(df.Y)
# Z_fft = to_fft(df.Z)

In [16]:
omega = 1/(1-tf.math.sqrt((tf.math.square(df.X) + tf.math.square(df.Y) + tf.math.square(df.Z))))
omega = np.array(omega)
omega = pd.DataFrame(omega)
omega

In [17]:
final = []
for i in range(24):
    check = df_output.loc[:,"Receiver_0"][5*i:5*(i+1)].astype(int)
    middle = [0]*30000
    for j in check:
        if j == -1:
            break
        middle[int(j)] =1
    final.extend(middle)
final = pd.DataFrame(final)
final

In [18]:
timestep = [i for i in range(720000)]
timestep = pd.DataFrame(timestep)

In [19]:
result = pd.concat([omega, final, timestep], axis = 1, )
result.columns = ["input", "output", "timestep"]
result

In [20]:
for index, row in result.iterrows():
    if row.output == 1:
        print(row.input, row.output, row.timestep)

In [21]:
plt.plot(result.input, result.timestep)